In [49]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Shared Variables

- 모든 노드에서 사용하기 위한 공유변수
- 특정 노드에서 값을 변경시키면 변경된 값이 모든 노드에게 전달되어야 한다.
- 공유변수로 지정한 값은 모든 노드에 중복되어 캐시된다.


- 반복적으로 사용해야하는 변수라면, 
  스파크의 노드는 네트워크를 통해 통신 하기 때문에 모든 노드에 중복 캐시하는 시스템적 비용보다  
  네트워크 과정에서 발생하는 오버헤드 비용이 더 많이 발생하게 된다. (계속 반복적으로 많이 사용해야 하는 변수 이기 떄문이다.)
  모든 노드에 중복 캐시하는 것이 더 좋을 수 있다는 의미이다.
  공유에서 쓰겠다는 의미(저장해놓고 계속 사용)
  컴퓨터에다가 캐쉬를 두고 (캐쉬라는 굉장히 빠른 메모리에다가 넣어놓고) 계속 사용하겠다.
  물리적으로는 각 컴퓨터의 캐쉬 메모리에 올려두겠다는 의미이다.

## Broadcast Variables

- 각 노드에 공유되는 읽기 전용 변수

In [1]:
# 학생별 수업카테고리코드로 지정되어있는 값을 카테고리 전체이름으로 변경한다고 가정 해보자

data = [("홍길동","DE"),
    ("이제동","DS"),
    ("김철수","DE"),
    ("변현재","WD")]

code_desc = {"DE":"Data Engineer", "DS":"Data Science", "WD":"Web Developer"}

student_rdd = sc.parellelize(data, 3)
student_rdd.collect()

In [2]:
# student_rdd 는 key-value 에 해당하는 pair rdd 이다.
# 약어(수업 카테고리 코드)로 된 수업 카테고리를 전체 카테고리명으로 변경하겠다.
# student_rdd 의 key 는? 이름 - 즉, 변경할 카테고리 코드는 value 이다.
# 즉 mapValues() 를 사용하면 된다. - value 를 넘겨 받아서 넘어온 value와 code_desc 의 key 와 매칭 후에 해당되는 key 의 값을 가지고 온다.
students_rdd.mapValues(lambda e : code_desc[e]).collect()

# Broadcast_variables 사용하기
- spark session 활용: 현재 수업 환경 spark 라는 객체변수로 제공
- spark session.SparkContext.broadcast(읽기 전용 공유변수도 사용할 값이나 변수)

In [ ]:
# Broadcast_variables 생성하기
# broadcast 함수를 사용해 생성하는 시점에 이미 SparkContext에 등록이 되고 객체 변수도 반환하게 됨
code_desc = {"DE":"Data Engineer", "DS":"Data Science", "WD":"Web Developer"}
broadcast_S = spark.session.SparkContext.broadcast(code_desc)
broadcast_S.value

In [ ]:
# 읽기전용 변수객체에 수정을 하면
broadcast_S.value['DE'] = 'process'
broadcast_S.value

# 삭제를 하면 
del(broadcast_S.value['DE'])
broadcast_S.value

#### broadcast 객체 자체에는 변경이나 삭제가 반영이 되지만
- sc에는 생성시점의 data가 그대로 유지가 된다. - 변경, 삭제가 전혀 반영되지 않는다. 

In [ ]:
student_rdd.mapValues(lambda e: broadcast_S[e]).collect()
# 아무일도 발생하지 않았다. - 그대로 함수가 진행된다.

## Accumulator

- 각 노드에 공유되는 누산기 함수
- 공유변수가 적용된 함수

In [ ]:
accum = sc.accumulator(0) # 누산기 함수 객체 변수 생성
rdd = spark.SparkContext.parallelize([1, 2, 3, 4, 5])
rdd.foreach(lambda x : accum.add(x))
print(accum.value)

In [3]:
# # accumulator를 사용하지 않는다면?
# a = 0

# # 모든 노드에서 발생하는 데이터 횟수를 확인해보자
# def change_cate(e):
#      a = a + 1
#      return broadcastStates.value[e]
    
# students_rdd.mapValues(lambda e : change_cate(e)).collect()

# # 횟수 확인
# # local variable 'a' referenced before assignment 발생
# a




In [4]:


# 모든 노드에서 발생하는 데이터 횟수를 확인해보자


# 횟수 확인
# local variable 'a' referenced before assignment 발생
